<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/notebooks/2021-0223w2v_onomatopea_projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語オノマトペ表現の単語ベクトル (word2vec) モデルによる理解

- date: 2021-0223
- author: 浅川伸一
- lincense: MIT license
- filename 2021kondo_onomatopea.ipynb

使い方の詳細は内部にコメントとして記述しています。
定義した射影空間，と <font color="teal" size="+1">歪んだ</font>射影空間への射影ベクトルを使った最近接語を表示します。

後半は河辺さんの卒業論文から使われた単語を上の射影行列で変換し，検索した単語を表示しています。




In [ ]:
!pwd

In [ ]:
import os
import sys
from scipy import stats
import pandas as pd
import numpy as np
np.set_printoptions(precision=2)

import jaconv  # ひらがなカタカナ変換用 `pip install jaconv` してください
# !pip install jaconv

#import matplotlib.pyplot as plt
#import japanize_matplotlib  # matplotlib の日本語表示
#%matplotlib inline


# 2021/Jan 近藤先生からいただいたオノマトペ辞典のデータ
# 著作権の問題が不明のため colab で利用可能データにしていませんのでご注意ください
# データ置き場
ccap_base = '/Users/asakawa/study/2021ccap/notebooks'

# xlsx ファイルでは読み込めないため xls ファイルに変換して利用
#onomatopea_excel = '日本語オノマトペ辞典4500より.xlsx'
onomatopea_excel = '日本語オノマトペ辞典4500より.xls'

# 実際のエクセルファイルの読み込み
onmtp2761 = pd.read_excel(os.path.join(ccap_base, onomatopea_excel), sheet_name='2761語')

# 読み込んだデータ確認のため印字
print(len(list(onmtp2761['オノマトペ'])), ': データ数')
first_n = 5  # 最初の first_n 個のデータをテスト表示
print(list(onmtp2761['オノマトペ'])[:first_n], ':最初の', first_n, 'データを表示')

2761 : データ数
['あーん', 'あくせく', 'あたふた', 'あっけらかん', 'あっはっは'] :最初の 5 データを表示


In [ ]:
# word2vec データの読み込み
# ローカルディスクから読み込むようになっています。colab でお使いの場合には
# 適宜変更してください
w2v_base = '/Users/asakawa/study/2016wikipedia/'
w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz'

In [ ]:
# word2vec の読み込み準備
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

In [ ]:
# 訓練済 word2vec，訓練データは wikipedia 全文  読み込みに時間がかかります
# 2つ上のセルと同様，ローカルドライブから読み込むようになっています。
w2v_base = '/Users/asakawa/study/2016wikipedia/'
w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz'
#w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg10_cbow.bin.gz'
#w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_sgns.bin.gz'
#w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg10_sgns.bin.gz'
asakawa_w2v_file = os.path.join(w2v_base, w2v_file)
asakawa_w2v = KeyedVectors.load_word2vec_format(asakawa_w2v_file, 
                                                encoding='utf-8', 
                                                unicode_errors='replace',
                                                binary=True) 
w2v = asakawa_w2v

In [ ]:
# オノマトペのうち，word2vec に登録があるかどうかを調査
kana_entries = []
kata_entries = []
count = 0
for word in list(onmtp2761['オノマトペ']):
    count += 1
    if word in w2v.vocab:
        kana_entries.append(word)
    else:
        # オノマトペが word2vec 項目にない場合はカタカナに変換する
        kata_w = jaconv.hira2kata(word)
        if kata_w in w2v.vocab:
            kata_entries.append(kata_w)
            #onmtp2761['オノマトペ'][word] = kata_w

print('There are', len(set(kana_entries)), 'kana onomatopea words in jawikipedia word2vec')
print('There are', len(set(kata_entries)), 'katakana onomatopea words in jawkipedia word2vec.')

# entries 
entries = list(set(kana_entries + kata_entries))
print('There are {} entries in total.'.format(len(entries)))

There are 893 kana onomatopea words in jawikipedia word2vec
There are 236 katakana onomatopea words in jawkipedia word2vec.
There are 1129 entries in total.


---

ここから下がシミュレーションの本体です。
---

In [ ]:
class w2v_subspace():
    """射影行列の表示
    少し内容を変更しました。クラス初期化時に引数 entries に任意の単語リストを入れると
    その単語リストで構成される部分空間を構成するように変更しました。
    
    また，結果呼び出し時に 引数 verbose=False にすると各エントリーについて，それぞれの
    射影行列 4 種とオリジナル word2vec による最近接語を topn にしてされた個数だけ入れた
    リストから構成される dict を返すようにしました
    
    
    # w2v_subspace():
    w2v: class w2v
        gensim 形式の訓練済 word2vec クラス
    entries: list
        部分空間を構成する単語のリスト
        word2vec に該当する単語がなければ無視する
        
    # _call_(targets, verbose):
    tarets: [str,]
        調べたい単語リスト
        例: targets=['トラ', 'イヌ', 'ペロペロ']
    topn: (int)
        最近接語数
    verbose: (bool)
        True: その都度結果を印字する
        Flase: 結果は dict 型データに格納して返す
    """
    
    def __init__(self, w2v=w2v, entries=None):
        """オノマトペリスト wordlist はエクセルファイルであることを仮定
        """
        self.w2v = w2v
        self.count = 0
        if entries != None:
            self.entries = wordlist
            self.count = len(self.entries)
        else:
            # オノマトペのうち，word2vec に登録があるかどうかを調査
            kana_entries, kata_entries = [], []
            for word in list(onmtp2761['オノマトペ']):
                self.count += 1
                
                if word in w2v.vocab:
                    # word2vec にエントリがある場合
                    kana_entries.append(word)
                else:
                    # オノマトペが word2vec 項目にない場合はカタカナに変換
                    kata_w = jaconv.hira2kata(word)
                    if kata_w in w2v.vocab:
                        # カタカナのエントリ存在する場合
                        kata_entries.append(kata_w)
        
            self.entries = list(set(kana_entries + kata_entries))

        # 射影行列 P を初期化    
        self.P = np.zeros((len(set(self.entries)), w2v.vector_size))
        
        # 射影空間の作成
        for i, x in enumerate(set(self.entries)):
            self.P[i] = np.copy(self.w2v[x])
            
        # 内積逆行列 (X^{T} X)^{-1} の作成
        invP = np.linalg.inv(np.dot(self.P, self.P.T))
        
        # 直交射影行列 Proj の作成
        self.Proj = np.dot(self.P.T, np.dot(invP, self.P))
        
        # 歪んだ射影行列の作成 2021年2月23日 近藤公久先生とのディスカッション
        # で解釈可能なら残しても良いということで使ってみる
        self.wProj = np.dot(self.P.T, self.P)

        # 直交補空間への射影行列の作成
        I = np.eye(len(self.Proj))
        self.C = I - self.Proj
        self.wC = I - self.wProj
        return

    def __call__(self, targets=['電車','ネコ','東京'], topn=5, verbose=True):
        ret = {}
        for target in targets:
            if not target in self.w2v:
                # もし word2vec にエントリが無かったらカタカナに変換
                k_target = jaconv.hira2kata(target)
                if k_target in self.w2v:
                    target = k_target
                else:
                    continue
                    
            # target 語の意味ベクトルを取得
            x = np.array(self.w2v[target])
            
            # 射影ベクトルを計算
            wPx = np.dot(self.wProj, x)
            wPx_list = [w[0] for w in list(self.w2v.similar_by_vector(wPx,topn=topn))]
            
            wPxc = np.dot(self.wC, x)
            wPxc_list = [w[0] for w in list(self.w2v.similar_by_vector(wPxc,topn=topn))]

            Px = np.dot(self.Proj, x)
            Px_list = [w[0] for w in self.w2v.similar_by_vector(Px,topn=topn)]

            Cx = np.dot(self.C, x)
            Cx_list = [w[0] for w in w2v.similar_by_vector(Cx,topn=topn)]
            
            w2v_list = [w[0] for w in list(self.w2v.similar_by_vector(x,topn=topn))]
            
            ret[target] = {
                'word2vec': w2v_list,
                'wProj': wPx_list,
                'wProj_c': wPxc_list,
                'Proj': Px_list,
                'Proj_c': Cx_list
            }

            if verbose:
                print('word2vec 最近隣語({0}):{1}'.format(target, w2v_list))
                print('         w付き射影ベクトル({0}):{1}'.format(target, wPx_list))
                print('        w付き補射影ベクトル({0}):{1}'.format(target, wPxc_list))
                print('          直交射影ベクトル({0}):{1}'.format(target, Px_list))
                print('         直交射影補ベクトル({0}):{1}'.format(target, Cx_list))
        
        if verbose:
            return
        else:
            return ret
        
P = w2v_subspace()
P(topn=7)


word2vec 最近隣語(電車):['電車', '車両', '気動車', '客車', '特急電車', '列車', '路面電車']
         w付き射影ベクトル(電車):['ゆったり', 'ぶらぶら', 'のんびり', '小走り', '散歩', 'ほのぼの', 'ぐるり']
        w付き補射影ベクトル(電車):['英政府', '中国国防部', 'Fayol', 'アダム・ベンフォード', 'シシンガマンガラジャ', 'オーサーシップ', '国防総省']
          直交射影ベクトル(電車):['73434', 'Fitiuta', 'エルンスト・ラドウィグ', 'Skeel', '17306', 'ヴァルター・リュティ', 'ウアチョン・ユナイテッド']
         直交射影補ベクトル(電車):['崇州市', 'smth', 'Lotfernrohr', 'リバリコバ', 'religioustolerance', 'tsuhanshinbun', '希京']
word2vec 最近隣語(ネコ):['ネコ', '猫', 'ウサギ', 'イヌ', 'ネズミ', 'クマ', 'キツネ']
         w付き射影ベクトル(ネコ):['うー', 'くん', 'ぺっ', 'ふー', 'ちゃん', 'くー', 'ぴょん']
        w付き補射影ベクトル(ネコ):['国内', 'SOMETIMES', 'Kavanaugh', '14項目', 'トゥイーギ・ラミレズ', '実績', '20項目']
          直交射影ベクトル(ネコ):['甲子園春夏連覇', 'Consolation', 'Batteer', 'クリップルド・インサイド', '玉祥', 'Zìzhìzhōu', 'エチオピアの歴史']
         直交射影補ベクトル(ネコ):['固形化', '22K', '鍛造', '大同江ビール', '鍛鉄', 'ヘルストマール', '日本製']
word2vec 最近隣語(東京):['東京', '大阪', '名古屋', '横浜', '京都', '銀座', '神戸']
         w付き射影ベクトル(東京):['東京', '大阪', '上野', 'ぶらり', '京都', '名古屋', '浅草']
        

In [ ]:
P(targets=['がたがた', 'ガタガタ', 'どきどき','ドキドキ', 'つるつる', 'ツルツル','すべすべ', 'スベスベ'])

word2vec 最近隣語(がたがた):['がたがた', 'Fantastico', 'ふん', 'すすり泣い', 'とうろうながし']
         w付き射影ベクトル(がたがた):['うー', 'ぺっ', 'ふー', 'たん', 'あー']
        w付き補射影ベクトル(がたがた):['国内', 'ベン・イェフダー', 'ヤロシ', '実績', 'トゥイーギ・ラミレズ']
          直交射影ベクトル(がたがた):['Consolation', '235種', '20101012', 'ミアセラ', 'アクセス制御リスト']
         直交射影補ベクトル(がたがた):['ストーイ・ワーナー', 'ジュノオ', '浅川書房', 'エイチアンドアイ', '山田幸五郎']
word2vec 最近隣語(ガタガタ):['ガタガタ', 'バタバタ', '絶叫', '物凄い', '震え']
         w付き射影ベクトル(ガタガタ):['よー', 'うー', 'あー', 'ぺっ', 'くん']
        w付き補射影ベクトル(ガタガタ):['国内', 'SOMETIMES', 'エイタム', 'Nジャマーキャンセラー', 'ヤロシ']
          直交射影ベクトル(ガタガタ):['préfecture', 'Consolation', '中国少年先鋒隊', '百家争鳴', 'ディファレント・フォー・ガールズ']
         直交射影補ベクトル(ガタガタ):['マルサン商店', 'タカトクトイス', '長谷川製作所', '今井科学', '模型メーカー']
word2vec 最近隣語(どきどき):['どきどき', 'ドキドキ', 'わくわく', 'ときめき', 'マジカル']
         w付き射影ベクトル(どきどき):['ぺっ', 'ふー', 'くん', 'ぴよ', 'うー']
        w付き補射影ベクトル(どきどき):['国内', '夫々', 'トゥイーギ・ラミレズ', 'ヤロシ', 'Kavanaugh']
          直交射影ベクトル(どきどき):['クリップルド・インサイド', 'ホワット・ユー・アー・ドゥーイング', '美しき麗しき日々', 'sinehime', 'kgr']
       

In [ ]:
P(targets=['電車','ネコ','男性'])

word2vec 最近隣語(電車):['電車', '車両', '気動車', '客車', '特急電車']
         w付き射影ベクトル(電車):['ゆったり', 'ぶらぶら', 'のんびり', '小走り', '散歩']
        w付き補射影ベクトル(電車):['英政府', '中国国防部', 'Fayol', 'アダム・ベンフォード', 'シシンガマンガラジャ']
          直交射影ベクトル(電車):['73434', 'Fitiuta', 'エルンスト・ラドウィグ', 'Skeel', '17306']
         直交射影補ベクトル(電車):['崇州市', 'smth', 'Lotfernrohr', 'リバリコバ', 'religioustolerance']
word2vec 最近隣語(ネコ):['ネコ', '猫', 'ウサギ', 'イヌ', 'ネズミ']
         w付き射影ベクトル(ネコ):['うー', 'くん', 'ぺっ', 'ふー', 'ちゃん']
        w付き補射影ベクトル(ネコ):['国内', 'SOMETIMES', 'Kavanaugh', '14項目', 'トゥイーギ・ラミレズ']
          直交射影ベクトル(ネコ):['甲子園春夏連覇', 'Consolation', 'Batteer', 'クリップルド・インサイド', '玉祥']
         直交射影補ベクトル(ネコ):['固形化', '22K', '鍛造', '大同江ビール', '鍛鉄']
word2vec 最近隣語(男性):['男性', '女性', '中年', '男女', '初老']
         w付き射影ベクトル(男性):['ベタベタ', 'かわいい', 'ちゃん', 'のんびり', '女の子']
        w付き補射影ベクトル(男性):['高Q', 'セントニゼイション', 'ヴォーパル', 'ツインバスターライフル', 'スペース・ローンチ・システム']
          直交射影ベクトル(男性):['第32回選抜高等学校野球大会', '00559', '070517', '000000029', '甲子園春夏連覇']
         直交射影補ベクトル(男性):['handycam', 'KRUN

In [ ]:
# 確認用，word2vec に登録されている単語を任意に n_test だけ調べて表示する
n_test = 5
for i in range(5):
    word = np.random.choice(list(w2v.vocab.keys()))
    P([word])


word2vec 最近隣語(59勝):['59勝', '54勝', '40勝', '44勝', '48勝']
         w付き射影ベクトル(59勝):['がっしり', 'すらり', '59勝', '42勝', 'どっしり']
        w付き補射影ベクトル(59勝):['ぼー', 'ぱー', 'みらくる', 'とらぶる', 'どんちゃん']
          直交射影ベクトル(59勝):['シュヴィェツォーフ', 'ジェ・ケンチェン・ジグメ・プンツォク・リンポチェ', 'GP', 'ヴェレナ・パラヴェル', 'マシン']
         直交射影補ベクトル(59勝):['13626', 'bethpage', 'ジョナサン・ヘレーラ', 'Suppiluliuma', 'Myomanji']
word2vec 最近隣語(麻生総理):['麻生総理', '国家公務員制度改革推進本部', '田中眞紀子', '河村建夫', '麻生首相']
         w付き射影ベクトル(麻生総理):['よー', 'ほっこり', 'ちょい', 'ぽんぽん', 'まちゃ']
        w付き補射影ベクトル(麻生総理):['国内', '少なくとも', '社', '夫々', '負っ']
          直交射影ベクトル(麻生総理):['保智', '有馬豊氏', '小笠原秀政', '足守藩', '山林芳則']
         直交射影補ベクトル(麻生総理):['グッスマ', 'アラブ首長国連邦の大統領', 'ɘ', 'العزيز', 'ストレイトディール']
word2vec 最近隣語(保健師助産師看護師養成所):['保健師助産師看護師養成所', '助産師学校', '保健師学校', '診療放射線技師養成所', '管理理容師']
         w付き射影ベクトル(保健師助産師看護師養成所):['とんとん', 'にゅー', 'もっち', 'ぴよ', 'ぎっ']
        w付き補射影ベクトル(保健師助産師看護師養成所):['Goad', 'だっ', 'あっ', 'Komanesky', '続い']
          直交射影ベクトル(保健師助産師看護師養成所):['諫草', '20110628', 'sportstory', '2009-07-21', 

In [ ]:
kawabe_tab1 = ['きーん', 'じわじわ', 'じんじん', 'すーすー', 'ぽかぽか']
P(kawabe_tab1)

word2vec 最近隣語(きーん):['きーん', '『～』）」', '六十面体', '柳家小蝠', 'しばん']
         w付き射影ベクトル(きーん):['たん', 'ふー', 'ぺっ', 'うー', 'ぎー']
        w付き補射影ベクトル(きーん):['国内', 'Kavanaugh', 'ベン・イェフダー', 'Maslov', '内約']
          直交射影ベクトル(きーん):['諫草', 'ICPC', 'ビクトリア・パブロビッチ', '心無き', '柊深夜']
         直交射影補ベクトル(きーん):['narikawa', 'emiclassics', '鉄道模型社', '井崎博之', 'ペクテル']
word2vec 最近隣語(じわじわ):['じわじわ', '一気に', 'ジワジワ', 'じりじり', '次々']
         w付き射影ベクトル(じわじわ):['ふわっ', 'よー', 'あー', 'ぺっ', 'くっ']
        w付き補射影ベクトル(じわじわ):['ベン・イェフダー', 'SOMETIMES', 'ヤロシ', '侶蘭', 'エイタム']
          直交射影ベクトル(じわじわ):['百家争鳴', '科学的実在論', 'エイモリー・B・ロビンス', '革命', '安楽県']
         直交射影補ベクトル(じわじわ):['ベタメタゾン', 'Ruocco', 'ウッシャーク', 'kohime', 'Stapleford']
word2vec 最近隣語(じんじん):['じんじん', '桜花爛漫', 'サクラサク', '希空', '歌謡曲だよ、人生は']
         w付き射影ベクトル(じんじん):['ふー', 'ぺっ', 'うー', 'ぎっ', 'ぴよ']
        w付き補射影ベクトル(じんじん):['国内', 'ベン・イェフダー', '米国', 'ヤロシ', 'Maslov']
          直交射影ベクトル(じんじん):['クリップルド・インサイド', 'Kannabe', '希望について', 'OverDriveTrain', '150505']
         直交射影補ベクトル(じんじん):['日本製', '歯磨', 'エモト', '電機

In [ ]:
kawabe_tab2 = (# 自然-天気
    'かんかん', 'はらはら', 'ごろごろ', 'ばらばら', 'ざーざー', 'ぱらぱら', 'さーっ',
    'びしゃびしゃ', 'ざーっ', 'ひゅー', 'さーさー', 'ひゅーひゅー', 'じりじり', 'びゅーびゅー',
    'そよそよ', 'びゅんびゅん',  'ばたばた', 'ぴゅんぴゅん')
P(kawabe_tab2)

word2vec 最近隣語(かんかん):['かんかん', 'むし', '蛙', '蟋蟀', 'ぶし']
         w付き射影ベクトル(かんかん):['たん', 'ぺっ', 'ふー', 'うー', 'どん']
        w付き補射影ベクトル(かんかん):['国内', 'ベン・イェフダー', '米国', 'Kavanaugh', '28日後']
          直交射影ベクトル(かんかん):['玖珠盆地', 'ICPC', 'IntServ', 'Xr', 'ヒャルティ・スケッギャソン']
         直交射影補ベクトル(かんかん):['emiclassics', 'トクホン', 'narikawa', 'クロックヴェルク', 'レイ・ヴァンプルー']
word2vec 最近隣語(はらはら):['はらはら', 'ドキドキ', 'ウキウキ', 'にらめっこ', 'ペッタン']
         w付き射影ベクトル(はらはら):['ぺっ', 'うー', 'ふー', 'くん', 'あー']
        w付き補射影ベクトル(はらはら):['国内', 'トゥイーギ・ラミレズ', 'SOMETIMES', '米国', 'ヤロシ']
          直交射影ベクトル(はらはら):['クリップルド・インサイド', 'sinehime', '150921', '中国少年先鋒隊', '甲子園春夏連覇']
         直交射影補ベクトル(はらはら):['narikawa', 'マランツ', 'マルサン商店', 'emiclassics', 'リーメント']
word2vec 最近隣語(ごろごろ):['ごろごろ', 'ゴロゴロ', 'ふかし', '転がっ', '昼寝']
         w付き射影ベクトル(ごろごろ):['うー', 'ふー', 'ぺっ', 'あー', 'くん']
        w付き補射影ベクトル(ごろごろ):['国内', 'ベン・イェフダー', 'ヤロシ', '米国', 'SOMETIMES']
          直交射影ベクトル(ごろごろ):['クリップルド・インサイド', 'エチオピアの歴史', 'Humanae', '中国少年先鋒隊', '水島聡']
         直交射影補ベクトル(ごろごろ):['アニテル', '樋屋

In [ ]:
kawabe_tab3 = (# 自然 火・土
    'がたがた', 'ぱりぱり', 'がんがん', 'べちょっ', 'ごつごつ', 'ぼーっ', 'さくさく',
    'ぼーぼー', 'さくっ', 'ぼこぼこ', 'じゃりじゃり', 'ぼっ', 'ぱちぱち')
P(kawabe_tab3)

word2vec 最近隣語(がたがた):['がたがた', 'Fantastico', 'ふん', 'すすり泣い', 'とうろうながし']
         w付き射影ベクトル(がたがた):['うー', 'ぺっ', 'ふー', 'たん', 'あー']
        w付き補射影ベクトル(がたがた):['国内', 'ベン・イェフダー', 'ヤロシ', '実績', 'トゥイーギ・ラミレズ']
          直交射影ベクトル(がたがた):['Consolation', '235種', '20101012', 'ミアセラ', 'アクセス制御リスト']
         直交射影補ベクトル(がたがた):['ストーイ・ワーナー', 'ジュノオ', '浅川書房', 'エイチアンドアイ', '山田幸五郎']
word2vec 最近隣語(ぱりぱり):['ぱりぱり', '優風', 'Xuse', '琴の若子', 'コダマナオコ']
         w付き射影ベクトル(ぱりぱり):['ぺっ', 'ふー', 'うー', 'あー', 'ぴよ']
        w付き補射影ベクトル(ぱりぱり):['国内', 'ベン・イェフダー', 'ヤロシ', 'トゥイーギ・ラミレズ', 'エイタム']
          直交射影ベクトル(ぱりぱり):['七転八起☆至上主義!', 'マネージコード', '風にうたえば', 'SHINING☆STAR', '項致荘']
         直交射影補ベクトル(ぱりぱり):['大同江ビール', 'マランツ', 'narikawa', '歯磨', 'ドル円']
word2vec 最近隣語(がんがん):['がんがん', 'もりもり', 'てっちゃん', 'ガオガオ', 'がんこちゃん']
         w付き射影ベクトル(がんがん):['ふー', 'ぺっ', 'うー', 'くん', 'たん']
        w付き補射影ベクトル(がんがん):['国内', 'ヤロシ', 'Kavanaugh', 'トゥイーギ・ラミレズ', '夫々']
          直交射影ベクトル(がんがん):['クリップルド・インサイド', '平魯区', '矢口真里とストローハット', 'SHINING☆STAR', 'KingdomBEST']
         直交射影補ベ

In [ ]:
kawabe_tab4 = (# 自然 水・液体
    'がばがば', 'ぱしゃっ', 'ころころ', 'ばちゃっ', 'ざばっ', 'びしゃっ', 'さらさら',
    'ぴしゃっ', 'ざぶん', 'びちゃっ', 'じゃーじゃー', 'びちゃびちゃ', 'じゃぶん',
    'びちょびちょ', 'じゃぼん', 'ぶくぶく', 'だぼだぼ', 'ぷくぷく', 'たぷんたぷん', 
    'べちゃべちゃ', 'ちゃぷちゃぷ', 'ぼたぼた', 'ちゅっ', 'ぽたぽた', 'ちょろちょろ', 
    'ぽつん', 'どくどく', 'ぼとぼと', 'どぼどぼ', 'ぽとん', 'どぼん', 'ぽとぽと')

kawabe_tab7 = {# 両方の意味をもつ高認知度の単語 20 語 自然分類ごと
    '温度': ('ぽかぽか',),
    '天気': ('ごろごろ', 'ざーっ', 'じりじり', 'ぱらぱら', 'ひゅー', 'ひゅーひゅー', 'びゅんびゅん'),
    '火・土':('ごつごつ', 'じゃりじゃり', 'ぱちぱち', 'べちょっ', 'ぼーぼー', 'ぼこぼこ'),
    '水・液体':('さらさら', 'ちょろちょろ', 'ぱしゃっ', 'びちょびちょ', 'べちゃべちゃ', 'ぽつん')}

kawabe_tab8 = (#表 8 実験に使用したオノマトペ 10 語
    'がたがた', 'じりじり', 'かんかん', 'ばたばた', 'ごろごろ', 'ぱちぱち', 'ざーっ', 'ぶくぶく', 
    'さくさく', 'ぼーっ')

kawabe_tab9 = (# 表 9 実験に使用した多義語 10 語
    'さっぱり', '青い', 'そろそろ', 'メジャー', '鋭い', '確か',
    '相当', '甘い', '一体', '勝手')


P.getProj(kawabe_tab8)

In [ ]:
# 2017年度 工学院大学卒業論文 異なる意味をもつオノマトペに対する共通イメージ, 氏名：河辺 紗耶

kawabe_tab1 = (#自然-温度
'きーん', 'じわじわ', 'じんじん', 'すーすー', 'ぽかぽか')

kawabe_tab2 = (# 自然-天気
'かんかん', 'はらはら', 'ごろごろ', 'ばらばら', 'ざーざー', 'ぱらぱら', 'さーっ',
'びしゃびしゃ', 'ざーっ', 'ひゅー', 'さーさー', 'ひゅーひゅー', 'じりじり', 'びゅーびゅー',
'そよそよ', 'びゅんびゅん',  'ばたばた', 'ぴゅんぴゅん')

kawabe_tab3 = (# 自然 火・土
'がたがた', 'ぱりぱり', 'がんがん', 'べちょっ', 'ごつごつ', 'ぼーっ', 'さくさく',
'ぼーぼー', 'さくっ', 'ぼこぼこ', 'じゃりじゃり', 'ぼっ', 'ぱちぱち')

kawabe_tab4 = (# 自然 水・液体
'がばがば', 'ぱしゃっ', 'ころころ', 'ばちゃっ', 'ざばっ', 'びしゃっ', 'さらさら',
'ぴしゃっ', 'ざぶん', 'びちゃっ', 'じゃーじゃー', 'びちゃびちゃ', 'じゃぶん',
'びちょびちょ', 'じゃぼん', 'ぶくぶく', 'だぼだぼ', 'ぷくぷく', 'たぷんたぷん', 
'べちゃべちゃ', 'ちゃぷちゃぷ', 'ぼたぼた', 'ちゅっ', 'ぽたぽた', 'ちょろちょろ', 
'ぽつん', 'どくどく', 'ぼとぼと', 'どぼどぼ', 'ぽとん', 'どぼん', 'ぽとぽと')

kawabe_tab7 = {# 両方の意味をもつ高認知度の単語 20 語 自然分類ごと
'温度': ('ぽかぽか',),
'天気': ('ごろごろ', 'ざーっ', 'じりじり', 'ぱらぱら', 'ひゅー', 'ひゅーひゅー', 'びゅんびゅん'),
'火・土':('ごつごつ', 'じゃりじゃり', 'ぱちぱち', 'べちょっ', 'ぼーぼー', 'ぼこぼこ'),
'水・液体':('さらさら', 'ちょろちょろ', 'ぱしゃっ', 'びちょびちょ', 'べちゃべちゃ', 'ぽつん')}

kawabe_tab8 = (#表 8 実験に使用したオノマトペ 10 語
'がたがた', 'じりじり', 'かんかん', 'ばたばた', 'ごろごろ', 'ぱちぱち', 'ざーっ', 'ぶくぶく', 
'さくさく', 'ぼーっ')

kawabe_tab9 = (# 表 9 実験に使用した多義語 10 語
'さっぱり', '青い', 'そろそろ', 'メジャー', '鋭い', '確か',
'相当', '甘い', '一体', '勝手')

kawabe_tab10 = {# 表10 実験 2 における刺激提示条件と例
    '同義条件': {
        '先行刺激': (('がたがた', '窓'), ('甘い', '果物')),
        'ターゲット':(('がたがた', '動く'), '甘い', '香り')},
    '異義条件': {
        '先行刺激': (('がたがた', '窓'), ('甘い', '果物')),
        'ターゲット':(('がたがた', '言う'), ('甘い', 'やり方'))},
    '無意味条件': {
        '先行刺激': (('がたがた', '窓'), ('甘い', '果物')),
        'ターゲット': (('がたがた', '干す'), ('甘い', '動物'))}}

def not_w2v(arg):
    n = 0
    if isinstance(arg, str):
        if not arg in w2v.vocab:
            _a = jaconv.hira2kata(arg)
            if not _a in w2v.vocab:
                n += 1
                print(_a, end=" ")
        return n
    elif isinstance(arg, tuple):
        for x in arg:
            n = not_w2v(x)
        return n
    elif isinstance(arg, dict):
        for x in arg.keys():
            print(x, end=": ")
            n = not_w2v(arg[x])
            print()
        return n


for x in [kawabe_tab1, kawabe_tab2, kawabe_tab3, 
          kawabe_tab4, kawabe_tab7, kawabe_tab9,
          kawabe_tab10
]:
    print(not_w2v(x))

#print(not_w2v(kawabe_tab7))


In [ ]:
words = ['うさぎ', 'あじさい', 'ワニ', '天ぷら', 'フクロウ', 'みかん', 
         'かご', 'バラ', 'へび', 'いちご', 'ソバ', 'おなか',
         'オレンジ', 'ハト', 'のど仏', '刺身', 'にわとり', 'スリコギ',
         'ぶどう', 'イチョウ', 'びわ', '手すり', '風車', '鋏',
         'ごはん', 'クジラ', 'タイ焼き', '靴べら', 'タンポポ', 'ヤギ',
         'エビ', 'かぶ', 'まんじゅう', 'リンゴ', 'タツノオトシゴ', 'レンコン']
P(words)


word2vec 最近隣語(うさぎ):['うさぎ', '猫', 'ウサギ', 'ねこ', '子猫']
         w付き射影ベクトル(うさぎ):['うー', 'ふー', 'ぺっ', 'くん', 'ちゃん']
        w付き補射影ベクトル(うさぎ):['国内', 'Kavanaugh', 'トゥイーギ・ラミレズ', '米国', '14項目']
          直交射影ベクトル(うさぎ):['甲子園春夏連覇', 'préfecture', 'LROC', 'リトルガール', 'ホワット・ユー・アー・ドゥーイング']
         直交射影補ベクトル(うさぎ):['歯磨', '固形化', '大同江ビール', '樋屋奇応丸', 'ニットー']
word2vec 最近隣語(あじさい):['あじさい', '紫陽花', 'アジサイ', 'つつじ', 'もみじ']
         w付き射影ベクトル(あじさい):['ふー', 'ぴよ', 'うー', 'ぺっ', 'たん']
        w付き補射影ベクトル(あじさい):['米国', '国内', 'アウラキ', 'ベン・イェフダー', 'しかしながら']
          直交射影ベクトル(あじさい):['シャステレ', '聖誕教会', '稲沢市長', 'IntServ', 'ジョエル・ヘイスティングス・メトカーフ']
         直交射影補ベクトル(あじさい):['茶川一郎', '武野藤介', '広田順', '勘繰', 'グランパパプロダクション']
word2vec 最近隣語(ワニ):['ワニ', 'ヘビ', 'トカゲ', 'サメ', 'カワウソ']
         w付き射影ベクトル(ワニ):['くん', 'うー', 'ちゃん', 'ぺっ', 'ふー']
        w付き補射影ベクトル(ワニ):['国内', 'SOMETIMES', 'Freundlich', 'バクー・イニシアティヴ', '14項目']
          直交射影ベクトル(ワニ):['Bowls', 'Consolation', 'Orgueil', '勝浦海上保安署', 'OpenStreetmap']
         直交射影補ベクトル(ワニ):['オームコ', 'ミラドライ', 'ヴァン・ストラテン', '

In [ ]:
kawabe_taba1=(#表 A1 予備実験に使用したオノマトペ','意味分類','自然-温度(9 語)
#自然-温度
'きーん', 'ぽかぽか', 'ぎんぎん', 'ぽかりぽかり', 'じわじわ', 'ぽっぽっ', 'じんじん', 'りん', 'すーすー')

print(P(kawabe_taba1))

word2vec 最近隣語(きーん):['きーん', '『～』）」', '六十面体', '柳家小蝠', 'しばん']
         w付き射影ベクトル(きーん):['たん', 'ふー', 'ぺっ', 'うー', 'ぎー']
        w付き補射影ベクトル(きーん):['国内', 'Kavanaugh', 'ベン・イェフダー', 'Maslov', '内約']
          直交射影ベクトル(きーん):['諫草', 'ICPC', 'ビクトリア・パブロビッチ', '心無き', '柊深夜']
         直交射影補ベクトル(きーん):['narikawa', 'emiclassics', '鉄道模型社', '井崎博之', 'ペクテル']
word2vec 最近隣語(ぽかぽか):['ぽかぽか', 'にこにこ', 'はらっぱ', 'わいわい', 'ほっこり']
         w付き射影ベクトル(ぽかぽか):['ふー', 'うー', 'ぺっ', 'ぴよ', 'とんとん']
        w付き補射影ベクトル(ぽかぽか):['国内', '米国', 'ヤロシ', 'ベン・イェフダー', 'トゥイーギ・ラミレズ']
          直交射影ベクトル(ぽかぽか):['美しき麗しき日々', '2005年ドイツ連邦議会選挙', 'クリップルド・インサイド', 'ムーラテスト', '希望について']
         直交射影補ベクトル(ぽかぽか):['narikawa', '固形化', 'グランパパプロダクション', 'ライカ', 'チヨタックス']
word2vec 最近隣語(ぎんぎん):['ぎんぎん', 'ぎらぎら', 'お日様', 'よりそい', '波間']
         w付き射影ベクトル(ぎんぎん):['ぺっ', 'ふー', 'うー', 'くん', 'ぴよ']
        w付き補射影ベクトル(ぎんぎん):['国内', 'ヤロシ', '米国', 'エイタム', '当局']
          直交射影ベクトル(ぎんぎん):['アイブロックス・スタジアム', 'スモーガー', 'ムーラテスト', 'tcab', 'Noordwijk']
         直交射影補ベクトル(ぎんぎん):['筋違い角', '前塚', 'エクラアニマル', '

In [ ]:
kawaba_taba2 = (# 表 A2 予備実験に使用したオノマトペ','意味分類','自然-天気(45 語)
#自然-天気
'かっ','ざんざら','びしゃびしゃ',
'かっか','ざんざ','ひゅー',
'からっ','しゃりしゃり','びゅー',
'からり','じりじり','ぴゅー',
'かんかん','すーっ','ひゅーひゅー',
'ごろごろ','そよそよ','びゅーびゅー',
'こんこん','そよ','ぴゅーぴゅー',
'ざーざー','そより','ひゅっ',
'さーっ','どしゃどしゃ','ぴゅっ',
'ざーっ','はたはた','びゅんびゅん',
'さーさー','ばたばた','ぴゅんぴゅん',
'ざざっ','ぱたぱた','ふー',
'ざっ','はらはら','ぶぉーっ',
'さやさや','ばらばら','ぶんぶん',
'さわさわ','ぱらぱら','みりり'
)

print(P(kawabe_tab2))

word2vec 最近隣語(かんかん):['かんかん', 'むし', '蛙', '蟋蟀', 'ぶし']
         w付き射影ベクトル(かんかん):['たん', 'ぺっ', 'ふー', 'うー', 'どん']
        w付き補射影ベクトル(かんかん):['国内', 'ベン・イェフダー', '米国', 'Kavanaugh', '28日後']
          直交射影ベクトル(かんかん):['玖珠盆地', 'ICPC', 'IntServ', 'Xr', 'ヒャルティ・スケッギャソン']
         直交射影補ベクトル(かんかん):['emiclassics', 'トクホン', 'narikawa', 'クロックヴェルク', 'レイ・ヴァンプルー']
word2vec 最近隣語(はらはら):['はらはら', 'ドキドキ', 'ウキウキ', 'にらめっこ', 'ペッタン']
         w付き射影ベクトル(はらはら):['ぺっ', 'うー', 'ふー', 'くん', 'あー']
        w付き補射影ベクトル(はらはら):['国内', 'トゥイーギ・ラミレズ', 'SOMETIMES', '米国', 'ヤロシ']
          直交射影ベクトル(はらはら):['クリップルド・インサイド', 'sinehime', '150921', '中国少年先鋒隊', '甲子園春夏連覇']
         直交射影補ベクトル(はらはら):['narikawa', 'マランツ', 'マルサン商店', 'emiclassics', 'リーメント']
word2vec 最近隣語(ごろごろ):['ごろごろ', 'ゴロゴロ', 'ふかし', '転がっ', '昼寝']
         w付き射影ベクトル(ごろごろ):['うー', 'ふー', 'ぺっ', 'あー', 'くん']
        w付き補射影ベクトル(ごろごろ):['国内', 'ベン・イェフダー', 'ヤロシ', '米国', 'SOMETIMES']
          直交射影ベクトル(ごろごろ):['クリップルド・インサイド', 'エチオピアの歴史', 'Humanae', '中国少年先鋒隊', '水島聡']
         直交射影補ベクトル(ごろごろ):['アニテル', '樋屋

In [ ]:
kawabe_taba3 = (#表 A3 予備実験に使用したオノマトペ','意味分類','自然-火・土(22 語)
#自然-火・土
'がたがた','ぱちぱち',
'がんがん','ぱりぱり',
'ごつごつ','ぶすぶす',
'さくさく','ぷすぷす',
'さくっ','べちょっ',
'さくりさくり','ぼーっ',
'ざっく','ぼーぼー',
'じゃくり','ぼーん',
'じゃりじゃり','ぼこぼこ',
'ずぶずぶ','ぼっ',
'ちろちろ','もー')

print(P(kawabe_taba3))


word2vec 最近隣語(がたがた):['がたがた', 'Fantastico', 'ふん', 'すすり泣い', 'とうろうながし']
         w付き射影ベクトル(がたがた):['うー', 'ぺっ', 'ふー', 'たん', 'あー']
        w付き補射影ベクトル(がたがた):['国内', 'ベン・イェフダー', 'ヤロシ', '実績', 'トゥイーギ・ラミレズ']
          直交射影ベクトル(がたがた):['Consolation', '235種', '20101012', 'ミアセラ', 'アクセス制御リスト']
         直交射影補ベクトル(がたがた):['ストーイ・ワーナー', 'ジュノオ', '浅川書房', 'エイチアンドアイ', '山田幸五郎']
word2vec 最近隣語(ぱちぱち):['ぱちぱち', 'ジンジン', 'チカチカ', 'あ〜', 'クツ']
         w付き射影ベクトル(ぱちぱち):['ぺっ', 'ふー', 'うー', 'あー', 'よー']
        w付き補射影ベクトル(ぱちぱち):['国内', 'ヤロシ', 'ベン・イェフダー', 'エイタム', 'SOMETIMES']
          直交射影ベクトル(ぱちぱち):['八七会議', 'ホワット・ユー・アー・ドゥーイング', 'クリップルド・インサイド', '百家争鳴', '七転八起☆至上主義!']
         直交射影補ベクトル(ぱちぱち):['narikawa', 'リーメント', '固形化', 'マランツ', 'emiclassics']
word2vec 最近隣語(がんがん):['がんがん', 'もりもり', 'てっちゃん', 'ガオガオ', 'がんこちゃん']
         w付き射影ベクトル(がんがん):['ふー', 'ぺっ', 'うー', 'くん', 'たん']
        w付き補射影ベクトル(がんがん):['国内', 'ヤロシ', 'Kavanaugh', 'トゥイーギ・ラミレズ', '夫々']
          直交射影ベクトル(がんがん):['クリップルド・インサイド', '平魯区', '矢口真里とストローハット', 'SHINING☆STAR', 'KingdomBEST']
   

In [ ]:
kawabe_taba4 = (#表 A4 予備実験に使用したオノマトペ	意味分類	自然-水・液体(55 語)
#自然-水・液体
'がばがば','ずっぷり','とぷんとぷん','しょろしょろ',
'がぼがぼ','だぼだぼ','どぼん','ぷくぷく',
'ころころ','たぷんたぷん','ばしゃっ','べちゃべちゃ',
'ざばっ','たぽたぽ','ぱしゃっ','ぼしゃっ',
'ざぶっ','ちゃぷちゃぷ','ばちゃっ','ぼたぼた',
'さらさら','ちゅっ','ぱちゃっ','ぽたり',
'ざぶん','ちょろちょろ','びしゃっ','ぽたん',
'ざんぶり','どーっ','ぴしゃっ','ぽたぽた',
'じくじく','どーどー','ひたひた','ぽつん',
'しゃーしゃー','とくとく','びちゃっ','ぼとぼと',
'じゃーじゃー','どくどく','ぴちゃっ','ぽとり',
'じゃぶん','どぶどぶ','びちゃびちゃ','ぽとん',
'じゃぼん','どぼどぼ','びちょびちょ','ぽとぽと',
'じゅくじゅく','どぶん','ぶくぶく')

print(P(kawabe_taba4))

word2vec 最近隣語(ガバガバ):['ガバガバ', '倉持茜', 'Waap', '薬学の時間', 'Pseudozizeeria']
         w付き射影ベクトル(ガバガバ):['ぺっ', 'ふー', 'よー', 'うー', 'ぎっ']
        w付き補射影ベクトル(ガバガバ):['国内', 'Kavanaugh', 'ヤロシ', '内約', 'エイタム']
          直交射影ベクトル(ガバガバ):['日大櫻丘', 'ヤサヴィー', '甲子園春夏連覇', 'ムーラテスト', 'マネージコード']
         直交射影補ベクトル(ガバガバ):['jcfia', 'emiclassics', 'narikawa', '）。/', 'スリチャパン']
word2vec 最近隣語(ダボダボ):['ダボダボ', 'ノースリーブ', 'ベルボトム', 'タンクトップ', 'タートルネック']
         w付き射影ベクトル(ダボダボ):['ふわふわ', 'ふわっ', 'ほっぺ', 'ふっくら', 'のんびり']
        w付き補射影ベクトル(ダボダボ):['エイタム', 'SOMETIMES', 'タッツォーリ', 'Freundlich', 'フォー・ユア・プレジャー']
          直交射影ベクトル(ダボダボ):['Humanae', 'ジョブネット', 'パイクビル', 'Relizane', '大埔県']
         直交射影補ベクトル(ダボダボ):['日鉄鉱業', '南部せんべい', '伸銅品', '横山賢二', 'Rinoie']
word2vec 最近隣語(どぼん):['どぼん', 'レッ', 'うぉー', 'りたーん', 'でん子']
         w付き射影ベクトル(どぼん):['ぺっ', 'ふー', 'うー', 'たん', 'きー']
        w付き補射影ベクトル(どぼん):['国内', 'トゥイーギ・ラミレズ', 'Kavanaugh', '夫々', '米国']
          直交射影ベクトル(どぼん):['ICPC', '08030138800', '20100404', '9784635922463', '玖珠盆地']
         直交射影補ベクトル(どぼん):

In [ ]:
kawabe_tab_b = (
('一万円','相当'),
('かんかん','お日様'),
('かんかん','叩く'),
('かんかん','照らす'),
('かんかん','色'),
('かんかん','鐘'),
('かんかん','食べる'),
('がたがた','動く'),
('がたがた','干す'),
('がたがた','文句'),
('がたがた','窓'),
('がたがた','窓'),
('がたがた','言う'),
('がたがた','漫画'),
('ごろごろ','寝る'),
('ごろごろ','急ぐ'),
('ごろごろ','紙'),
('ごろごろ','親父'),
('ごろごろ','雷'),
('ごろごろ','響く'),
('さくさく','かじる'),
('さくさく','リンゴ'),
('さくさく','作業'),
('さくさく','寝る'),
('さくさく','進める'),
('さくさく','食器'),
('さっぱり','不明'),
('さっぱり','味'),
('さっぱり','服'),
('さっぱり','本'),
('さっぱり','気分'),
('さっぱり','難しい'),
('ざーっ','夕立'),
('ざーっ','座る'),
('ざーっ','目次'),
('ざーっ','見る'),
('ざーっ','降る'),
('ざーっ','電気'),
('じりじり','ネクタイ'),
('じりじり','寒い'),
('じりじり','目覚まし'),
('じりじり','締め切り'),
('じりじり','迫る'),
('じりじり','鳴る'),
('そろそろ','1 年'),
('そろそろ','出発'),
('そろそろ','動いた'),
('そろそろ','星'),
('そろそろ','歩いた'),
('そろそろ','硬い'),
('ばたばた','なびく'),
('ばたばた','仕事'),
('ばたばた','国旗'),
('ばたばた','慌ただしい'),
('ばたばた','破れる'),
('ばたばた','趣味'),
('ぱちぱち','はじく'),
('ぱちぱち','まばたきする'),
('ぱちぱち','怒る'),
('ぱちぱち','火花'),
('ぱちぱち','目'),
('ぱちぱち','財布'),
('ぶくぶく','お腹'),
('ぶくぶく','休む'),
('ぶくぶく','太る'),
('ぶくぶく','座る'),
('ぶくぶく','泡'),
('ぶくぶく','湧き出る'),
('ぼーっ','携帯'),
('ぼーっ','汽笛'),
('ぼーっ','焦る'),
('ぼーっ','立ちすくむ'),
('ぼーっ','頭'),
('ぼーっ','鳴らす'),
('クラス','一体'),
('チーム','一体'),
('メジャー','作家'),
('メジャー','使う'),
('メジャー','商品'),
('メジャー','測る'),
('メジャー','痛い'),
('メジャー','箱'),
('一体','どこ'),
('一体','ノート'),
('一体','何'),
('一体','定規'),
('勝手','する'),
('勝手','信用'),
('勝手','分かる'),
('勝手','指す'),
('勝手','言う'),
('勝手','違う'),
('大学生','相当'),
('甘い','やり方'),
('甘い','乗る'),
('甘い','動物'),
('甘い','果物'),
('甘い','考え'),
('甘い','香り'),
('相当','危ない'),
('相当','嬉しい'),
('相当','耳'),
('相当','足'),
('確か','あの時'),
('確か','情報'),
('確か','技術'),
('確か','昨年'),
('確か','水'),
('確か','貼る'),
('鋭い','かご'),
('鋭い','刃物'),
('鋭い','勘'),
('鋭い','指摘'),
('鋭い','牙'),
('鋭い','髪'),
('青い','口'),
('青い','尻'),
('青い','海'),
('青い','空'),
('青い','考え'),
('青い','舌'),
('１万円','相当')
)

from itertools import chain

P(list(chain(*kawabe_tab_b)))


word2vec 最近隣語(相当):['相当', '該当', '匹敵', '存在', '付随']
         w付き射影ベクトル(相当):['閉口', 'ずんぐり', 'がっしり', 'さっぱり', 'おっとり']
        w付き補射影ベクトル(相当):['高Q', 'ビジネススキーム', 'タイムリーマーチ', 'バーコードシステム', '西澤桂']
          直交射影ベクトル(相当):['テラトルニス', ']]〜\u3000[[', 'ヒラタムシ', '三遊亭楽京', '桂さん都']
         直交射影補ベクトル(相当):['ティールロヴァー', '伏尾町', 'Taon', 'ウエスレー', 'Grimms']
word2vec 最近隣語(かんかん):['かんかん', 'むし', '蛙', '蟋蟀', 'ぶし']
         w付き射影ベクトル(かんかん):['たん', 'ぺっ', 'ふー', 'うー', 'どん']
        w付き補射影ベクトル(かんかん):['国内', 'ベン・イェフダー', '米国', 'Kavanaugh', '28日後']
          直交射影ベクトル(かんかん):['玖珠盆地', 'ICPC', 'IntServ', 'Xr', 'ヒャルティ・スケッギャソン']
         直交射影補ベクトル(かんかん):['emiclassics', 'トクホン', 'narikawa', 'クロックヴェルク', 'レイ・ヴァンプルー']
word2vec 最近隣語(お日様):['お日様', '夏空', '青い空', '夜空', '輝く夜']
         w付き射影ベクトル(お日様):['ぺっ', 'うー', 'ふー', 'ぴよ', 'あー']
        w付き補射影ベクトル(お日様):['国内', '米国', 'ヤロシ', 'ベン・イェフダー', 'トゥイーギ・ラミレズ']
          直交射影ベクトル(お日様):['ホワット・ユー・アー・ドゥーイング', '150921', 'クリップルド・インサイド', 'Humanae', 'Nansatsu']
         直交射影補ベクトル(お日様):['emiclassics', 'narikawa', 'アニテ

In [ ]:
# usage of gensim fasttext take a look at `https://towardsdatascience.com/word-embedding-with-word2vec-and-fasttext-a209c1d3e12c`
from gensim.models import FastText as gensim_ft
#help(gensim_ft.load)
gensim_base = '/Users/asakawa/study/2019facebookresearch_fastText.git'
#gensim_base = '/Users/asakawa/study/2020ccap/notebooks/models'
ft_e = gensim_ft()
ft_e.load_binary_data(os.path.join(gensim_base, 'cc.en.300.bin'))
#gensim_ft(wv=

In [ ]:
help(gensim_ft.load_binary_data)

In [ ]:
#ft_e = KeyedVectors.load_word2vec_format(os.path.join(gensim_base, 'cc.en.300.bin'),
ft_e = gensim_ft.load_binary_data(os.path.join(gensim_base, 'cc.en.300.bin'))


In [ ]:
#print('bus' in ft_e.wv)
print(ft_e.wv.most_similar(positive='cat', topn=10))
print(ft_e.wv.similarity("night", "nights"))
print(len(ft_e.wv.vocab))
#help(ft_e.wv.most_similar)